<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Screen_time_firebase_monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides the codes to check the progress of each participant, including two main functions:
1. Check the last update of screen events from each participant
2. Generate a list of emails (to be copied), to whom we should send a reminder email

# Preparation

Mount Google Drive and set up file path

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the Google Drive path, depending who is running the notebook

In [2]:
#Jian
google_drive_data_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Data'
google_drive_key_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Key'

In [ ]:
#Subin
# google_drive_data_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'
# google_drive_key_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

# Firebase database

## Set up link

In [5]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import pandas as pd

In [6]:
# Don't re-run this cell unless the kernel has been restarted
cred = credentials.Certificate(google_drive_key_path+'/timer-42ad2-firebase-adminsdk-4r7oj-2c373565f2.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://timer-42ad2-default-rtdb.firebaseio.com'
})

In [7]:
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

## Response data

In [8]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

def firebase_to_csv_log_Screen(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()

        # Process each host under the pid
        for host, events in pid_data.items():
            if isinstance(events, list):
                for index, event in enumerate(events):
                    if event is not None and isinstance(event, dict):
                        row = {
                            'pid': pid,
                            'host': host,
                            'eventLabel': index,
                            'startTime': event.get('startTime'),
                            'endTime': event.get('endTime')
                        }
                        transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    if len(transformed_data) > 0:
        df = df[['pid', 'host', 'eventLabel', 'startTime', 'endTime']]
    df.to_csv(output_file, index=False)

def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        pid = pid.lower()

        # Process each host under pid
        for host, host_data in pid_data.items():
            # If host_data is a list, enumerate through it
            if isinstance(host_data, list):
                for survey_id, survey_data in enumerate(host_data):
                    if survey_data is not None:
                        timestamp = get_timestamp(survey_data)
                        if timestamp:
                            meta_data.append({
                                'pid': pid,
                                'host': host,
                                'surveyID': str(survey_id),
                                'timestamp': timestamp
                            })

                            responses = get_responses(survey_data)
                            if responses:
                                responses_data.extend(process_responses(responses, pid, host, str(survey_id)))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    if len(meta_data) > 0:
        meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    if len(responses_data) > 0:
        first_cols = ['pid', 'host', 'surveyID']
        q_cols = [col for col in responses_df.columns if col.startswith('q_')]
        q_cols.sort(key=lambda x: int(x.split('_')[1]))
        responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

In [9]:
responses_to_csvs('survey_meta.csv', 'survey_responses.csv')
survey_meta_data = pd.read_csv('/content/survey_meta.csv')
print("The shape of the survey_meta_data is " + str(survey_meta_data.shape))
survey_meta_data.to_csv(google_drive_data_path + '/survey_meta.csv', index=False)
survey_responses_data = pd.read_csv('/content/survey_responses.csv')
print("The shape of the survey_responses_data is " + str(survey_responses_data.shape))
survey_responses_data.to_csv(google_drive_data_path + '/survey_responses.csv', index=False)

The shape of the survey_meta_data is (22, 4)
The shape of the survey_responses_data is (22, 26)


## Screen events

In [10]:
def firebase_to_csv_log_Screen(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    # dictionary structure
    for pid, pid_data in data.items():
        pid = pid.lower()

        # Skip if pid_data is not a dictionary (to handle old format entries)
        if not isinstance(pid_data, dict):
            continue

        # Process each host under the pid
        for host, events in pid_data.items():
            if not isinstance(events, list):
                continue

            for index, event in enumerate(events, start=0):
                if event is None:
                    continue
                if isinstance(event, dict):
                    row = {
                        'pid': pid,
                        'host': host,
                        'eventLabel': index,
                        'startTime': event.get('startTime'),
                        'endTime': event.get('endTime')
                    }
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    if len(transformed_data) > 0:
        df = df[['pid', 'host', 'eventLabel', 'startTime', 'endTime']]
    df.to_csv(output_file, index=False)

In [11]:
firebase_to_csv_log_Screen('/screen_events','screen_events.csv')
screen_events_data = pd.read_csv('/content/screen_events.csv')
print("The shape of the screen_events_data is " + str(screen_events_data.shape))
screen_events_data.to_csv(google_drive_data_path + '/screen_events.csv', index=False)

The shape of the screen_events_data is (2182, 5)


## Device info

In [12]:
# this function works for device and timezone infor
def firebase_to_csv(ref_path, output_file):
    # Get reference to device_info
    ref = db.reference(ref_path)
    data = ref.get()

    transformed_data = []

    # Transform the nested structure
    for pid, pid_data in data.items():
        pid = pid.lower()

        if isinstance(pid_data, str):
            # Case where timezone is directly under pid
            row = {
                'pid': pid,
                'host': pd.NA,  # or None
                'timezone': pid_data
            }
            transformed_data.append(row)
        else:
            # Case where pid has host-timezone pairs
            for host, timezone in pid_data.items():
                row = {
                    'pid': pid,
                    'host': host,
                    'timezone': timezone
                }
                transformed_data.append(row)

    # Convert to DataFrame
    df = pd.DataFrame(transformed_data)

    # Ensure pid and host are the first columns
    cols = ['pid', 'host'] + [col for col in df.columns if col not in ['pid', 'host']]
    df = df[cols]

    # Save to CSV
    df.to_csv(output_file, index=False)

In [13]:
firebase_to_csv('/device_info','device_info.csv')
device_data = pd.read_csv('/content/device_info.csv')

# First let's convert the string representation of dictionary to actual dictionary
device_data0 = device_data.copy()
device_data0['timezone'] = device_data0['timezone'].apply(eval)

# Now extract each field into its own column
device_data0['Base'] = device_data0['timezone'].apply(lambda x: x.get('Base'))
device_data0['Brand'] = device_data0['timezone'].apply(lambda x: x.get('Brand'))
device_data0['Model'] = device_data0['timezone'].apply(lambda x: x.get('Model'))

# If you want to drop the original timezone column, you can use:
device_data = device_data0.drop('timezone', axis=1)

print("The shape of the device_data is " + str(device_data.shape))
device_data.to_csv(google_drive_data_path + '/device_info.csv', index=False)

The shape of the device_data is (23, 5)


## Timezone info

In [14]:
firebase_to_csv('/timezones','timezones.csv')
timezones_data = pd.read_csv('/content/timezones.csv')
print("The shape of the timezones_data is " + str(timezones_data.shape))
timezones_data.to_csv(google_drive_data_path + '/timezones.csv', index=False)

The shape of the timezones_data is (23, 3)


## User states

This data was introduced in an update of the app, so it only exist in some of the participants' data.

In [15]:
firebase_to_csv('/user_states','user_states.csv')
user_states = pd.read_csv('/content/user_states.csv')

In [16]:
# First convert the string representation of dictionary to actual dictionary
user_states['timezone'] = user_states['timezone'].apply(eval)

# Extract each field into its own column
user_states['day_count'] = user_states['timezone'].apply(lambda x: x.get('day_count'))
user_states['survey1_completed'] = user_states['timezone'].apply(lambda x: x.get('survey1_completed'))
user_states['survey2_completed'] = user_states['timezone'].apply(lambda x: x.get('survey2_completed'))
user_states['survey3_completed'] = user_states['timezone'].apply(lambda x: x.get('survey3_completed'))
user_states['survey4_completed'] = user_states['timezone'].apply(lambda x: x.get('survey4_completed'))
user_states['tutorial_completed'] = user_states['timezone'].apply(lambda x: x.get('tutorial_completed'))

# Drop the original timezone column
user_states = user_states.drop('timezone', axis=1)

# Display the result
print("The shape of the user_states is " + str(user_states.shape))
user_states.to_csv(google_drive_data_path + '/user_states.csv', index=False)


The shape of the user_states is (20, 8)


## Setting changes

Data of setting changes will come in the third week. Don't run this chuck of code early than that, it will return just error.



In [ ]:
def firebase_to_csv_log_setting (ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    # dictionary structure
    for pid, changes in data.items():  # 'pid' = key
        if isinstance(changes, list):
            for index, change in enumerate(changes, start=0):  #event_label_start:1
                if change is None:
                    continue
                if isinstance(change, dict):
                    row = {'pid': pid,
                        'eventLabel': index,
                        'newValue' : change.get('newValue'),
                        'setting': change.get('setting'),
                        'timestamp': change.get('timestamp')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'newValue', 'setting', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_setting('/settings_change_logs','settings_change_logs.csv')
settings_change_logs_data = pd.read_csv('/content/settings_change_logs.csv')
print("The shape of the settings_change_logs_data is " + str(settings_change_logs_data.shape))
settings_change_logs_data.to_csv(google_drive_data_path + '/settings_change_logs.csv', index=False)

The shape of the settings_change_logs_data is (21, 4)


## Ringer

Data of ringer changes will come in the third week. Don't run this chuck of code early than that, it will return just error.

In [ ]:
def firebase_to_csv_log_mode(ref_path, output_file):
    ref = db.reference(ref_path)
    data = ref.get()
    transformed_data = []

    for pid, middle_levels in data.items():
        pid = pid.lower()
        for middle_key, sub_changes in middle_levels.items():
            for index, event in enumerate(sub_changes):
                if isinstance(event, dict):
                    row = {'pid': pid,'host':middle_key,'label': str(index),
                        'mode': event.get('mode'),
                        'timestamp': event.get('timestamp')}
                    transformed_data.append(row)

    df = pd.DataFrame(transformed_data)
    df = df[['pid', 'host', 'label', 'mode', 'timestamp']]
    df.to_csv(output_file, index=False)

In [ ]:
firebase_to_csv_log_mode('/ringer_mode_events','ringer_mode_events.csv')
ringer_mode_events_data = pd.read_csv('/content/ringer_mode_events.csv')
print("The shape of the ringer_mode_events_data is " + str(ringer_mode_events_data.shape))
ringer_mode_events_data.to_csv(google_drive_data_path + '/ringer_mode_events.csv', index=False)

## Counters

We don't actually need top analyze the counter data

In [ ]:
# def firebase_to_csv_counters(ref_path, output_file):
#     ref = db.reference(ref_path)
#     data = ref.get()
#     transformed_data = []
#     for pid, value in data.items():
#         pid = pid.lower()
#         # Case 1: only numeric
#         if isinstance(value, int):
#             transformed_data.append({'pid': pid, 'screen_event_count': value})

#         # Case 2: dictionary => selecting only value
#         elif isinstance(value, dict):
#             numeric_values = [v for v in value.values() if isinstance(v, int)]
#             if numeric_values:
#                 total = sum(numeric_values)
#                 transformed_data.append({'pid': pid, 'screen_event_count': total})

#     df = pd.DataFrame(transformed_data)
#     df = df[['pid', 'screen_event_count']]
#     df.to_csv(output_file, index=False)


# firebase_to_csv_counters('/ringer_event_counters','ringer_event_counters.csv')
# ringer_event_counters_data0 = pd.read_csv('/content/ringer_event_counters.csv')
# print(ringer_event_counters_data0.head())

# firebase_to_csv_counters('/screen_event_counters','screen_event_counters.csv')
# screen_event_counters_data0 = pd.read_csv('/content/screen_event_counters.csv')
# print(screen_event_counters_data0.head())

# firebase_to_csv_counters('/settings_change_counters','settings_change_counters.csv')
# settings_change_counters_data0 = pd.read_csv('/content/settings_change_counters.csv')
# print(settings_change_counters_data0.head())

# Paricipant management

## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

### Read in the meta data

In [17]:
# read in the data
import pytz
from datetime import datetime

survey_data = pd.read_csv(google_drive_data_path + '/survey_meta.csv')
survey_data['submitDate'] = pd.to_datetime(survey_data['timestamp'].map(convert_unix_to_date))

# Pivot the dataframe to create survey columns
survey_meta_pivoted = survey_data.pivot(index=['pid','host'], columns='surveyID', values='submitDate')

# Rename the columns
survey_meta_pivoted.columns = ['survey' + str(col) for col in survey_meta_pivoted.columns]

# Reset the index to make pid a column again
survey_meta_pivoted = survey_meta_pivoted.reset_index()


# Get today's date once
ny_tz = pytz.timezone('America/New_York')
today = pd.Timestamp.now(tz=ny_tz).normalize().date()

# Fixed date operations
survey_meta_pivoted['survey1'] = pd.to_datetime(survey_meta_pivoted['survey1']).dt.date
survey_meta_pivoted['real_day_count'] = np.array([today - d for d in survey_meta_pivoted['survey1']], dtype='timedelta64[D]').astype(int) + 1

def calculate_app_day_count(row):
   required_columns = ['survey2', 'survey3', 'survey4']
   row = row.reindex([*row.index, *required_columns]).fillna(pd.NA)

   today = pd.Timestamp.now(tz=ny_tz)
   s1 = pd.to_datetime(row['survey1']).tz_localize(ny_tz)
   days_since_s1 = (today - s1).days + 1

   if pd.notna(row['survey4']):
       return 0
   if pd.isna(row['survey2']):
       return min(8, days_since_s1)

   s2 = pd.to_datetime(row['survey2']).tz_localize(ny_tz)
   delay1 = max(0, (s2 - s1).days - 7)

   if pd.isna(row['survey3']):
       return min(15, days_since_s1 - delay1)

   s3 = pd.to_datetime(row['survey3']).tz_localize(ny_tz)
   delay2_threshold = 8 if (s2 - s1).days == 6 else 7
   delay2 = max(0, (s3 - s2).days - delay2_threshold)

   return min(22, days_since_s1 - delay1 - delay2)


print("The shape of the dataframe is: ", survey_meta_pivoted.shape)

survey_meta_pivoted['app_day_count'] = survey_meta_pivoted.apply(calculate_app_day_count, axis=1)

survey_meta_pivoted = (survey_meta_pivoted
   .assign(pid=lambda x: x['pid'].str.upper())
   .rename(columns={'pid': 'PID'}))


The shape of the dataframe is:  (22, 4)


### Read the email info and merge

Before running the cell below, make sure the Participants.xlsx is up to date.

In [18]:
participant_data = pd.read_excel(google_drive_data_path + '/Participants.xlsx')

survey_meta_manager = pd.merge(survey_meta_pivoted, participant_data, on='PID', how='right')

# Sort by survey1
survey_meta_manager = survey_meta_manager.sort_values(by='survey1')

# Drop the specified columns
survey_meta_manager = survey_meta_manager.drop(columns=['host', 'real_day_count'], errors='ignore')

## Check latest update

In [19]:
# Read and process screen data in one chain
latest_times = (pd.read_csv(google_drive_data_path + '/screen_events.csv')
   .groupby('pid')['endTime']
   .max()
   .reset_index()
   .assign(lastUpdate=lambda x: x['endTime'].map(lambda y: convert_unix_to_date(y, return_date=False)))
   .drop(columns=['endTime'])
   .assign(PID=lambda x: x['pid'].str.upper())
   .drop(columns=['pid']))

# Read and process timezone data in one chain
timezone_data = (pd.read_csv(google_drive_data_path + '/timezones.csv')
   .assign(PID=lambda x: x['pid'].str.upper())
   .drop(columns=['pid'])
   [['PID', 'timezone']])

# Combine all merges and data cleaning in one chain
survey_meta_manager = (survey_meta_manager
   .merge(latest_times, on='PID', how='left')
   .merge(timezone_data, on='PID', how='left')
   .assign(lastUpdate=lambda x: x['lastUpdate'].astype(str).replace('NaT', ''))
   .fillna(''))

Update lastUpdate with Timezone

In [20]:
import pandas as pd
from datetime import datetime
import pytz

def convert_to_local_time(row):
    # Skip rows where either lastUpdate or timezone is missing
    if pd.isna(row['lastUpdate']) or pd.isna(row['timezone']):
        return pd.NaT

    try:
        # Parse the timestamp
        dt = pd.to_datetime(row['lastUpdate'])
        if pd.isna(dt):
            return pd.NaT

        # Get source timezone (EST/EDT)
        source_tz = pytz.timezone('America/New_York')

        # Localize the datetime
        dt_source = source_tz.localize(dt.tz_localize(None))

        # Convert to target timezone
        target_tz = pytz.timezone(row['timezone'])
        dt_target = dt_source.astimezone(target_tz)

        return dt_target

    except (ValueError, AttributeError):
        return pd.NaT

# Apply the conversion
survey_meta_manager['local_time'] = survey_meta_manager.apply(convert_to_local_time, axis=1)

# Replace "NaT" with empty strings in the 'lastUpdate' column
survey_meta_manager['local_time'] = survey_meta_manager['local_time'].astype(str).replace('NaT', '')

In [21]:
def calculate_time_lapse(last_update):
    if pd.isna(last_update):
        return None

    # Get current time in EST (since lastUpdate is in EST)
    est = pytz.timezone('America/New_York')
    now = datetime.now(est)

    try:
        # Parse lastUpdate with timezone info (-05:00 is already in the data)
        last_update = pd.to_datetime(last_update, utc=True)
        last_update = last_update.astimezone(est)

        # Calculate time difference
        time_diff = now - last_update

        # Convert to hours and minutes
        total_minutes = time_diff.total_seconds() / 60
        hours = int(total_minutes // 60)
        minutes = int(total_minutes % 60)

        # Format as HH:MM
        return f"{hours:02d}:{minutes:02d}"
    except:
        return None

# Calculate time lapse for each row
survey_meta_manager['time_lapse'] = survey_meta_manager['lastUpdate'].apply(calculate_time_lapse)

# prompt: Rename "nick name":"nick_name", "timezone":"time_zone", "local_time":"last_update_local","time_lapse":"lapse_last_update"

# Rename the columns
survey_meta_manager = survey_meta_manager.rename(columns={
    "survey1": "start_date",
    "nick name": "nick_name",
    "timezone": "time_zone",
    "local_time": "last_update_local",
    "time_lapse": "lapse_last_update"
})

# prompt: reorder the columns: PID, Email, nick_name, start_date, app_day_count, last_update_local, lapse_last_update

# Reorder the columns
new_column_order = ['PID', 'Email', 'nick_name', 'start_date', 'app_day_count', 'last_update_local', 'lapse_last_update']
survey_meta_manager = survey_meta_manager[new_column_order]

survey_meta_manager.to_csv(google_drive_data_path + '/survey_meta_manager.csv', index=False)

Creating a status

In [30]:
import pandas as pd
from datetime import datetime, time

# Read the CSV
df = pd.read_csv(google_drive_data_path + '/survey_meta_manager.csv')
# saved_status = pd.read_excel(google_drive_data_path + '/status_saved.xlsx')
# df = df.merge(saved_status, on='Email', how='left')

# Initialize status column with "not yet started" ONLY where status < 4 or NaN
mask_initial = (df['status'] < 4) | (df['status'].isna())
df.loc[mask_initial, 'status'] = 1

# Convert lapse_last_update to datetime.time objects where possible
def convert_time(x):
    try:
        if pd.notna(x):
            hours, minutes = map(int, x.split(':'))
            return time(hours, minutes)
    except:
        return pd.NaT

df['lapse_time'] = df['lapse_last_update'].apply(convert_time)

# Set status based on conditions, but only where status < 4 or NaN
cutoff_time = time(12, 0)
mask_eligible = (df['status'] < 4) | (df['status'].isna())

# 1. If lapse_last_update < 12:00, status = "started"
mask_started = df['lapse_time'].apply(lambda x: pd.notna(x) and isinstance(x, time) and x < cutoff_time)
df.loc[mask_eligible & mask_started, 'status'] = 2

# 2. If start_date is not empty but lapse_last_update is empty or ≥ 12:00
mask_tech_issue = (pd.notna(df['start_date'])) & (~mask_started)
df.loc[mask_eligible & mask_tech_issue, 'status'] = 3

# Drop the temporary lapse_time column
df = df.drop('lapse_time', axis=1)
df[['Email','status']].to_excel(google_drive_data_path + '/status_saved.xlsx', index=False)

In [32]:
# Create status-note mapping dictionary from the CSV data
status_mapping = {
    0: 'to invite',
    1: 'invited',
    2: 'started',
    3: 'data paused',
    4: 'completed',
    5: 'dropped out',
    9: 'tester'
}

# Delete 'note_yesterday' column if it exists
if 'note_yesterday' in df.columns:
    df = df.drop(columns=['note_yesterday'])

# Rename 'note' column to 'note_yesterday' if it exists
if 'note' in df.columns:
    df = df.rename(columns={'note': 'note_yesterday'})

# Create a new 'note' column and initialize it with NaN
df['note'] = float('nan')

# Update the new 'note' column using the status mapping
df.loc[df['note'].isna(), 'note'] = df.loc[df['note'].isna(), 'status'].map(status_mapping)
df.to_csv(google_drive_data_path + '/survey_meta_manager.csv', index=False)

<ipython-input-32-b26f8a6b0097>:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['data paused' 'started' 'started' 'dropped out' 'started' 'started'
 'started' 'data paused' 'started' 'data paused' 'started' 'started'
 'started' 'started' 'started' 'started' 'started' 'started' 'started'
 'started' 'started' 'tester' 'tester' 'started' 'invited' 'invited'
 'invited' 'invited' 'invited' 'invited' 'invited' 'invited' 'invited']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['note'].isna(), 'note'] = df.loc[df['note'].isna(), 'status'].map(status_mapping)


## further process

In [ ]:
# Read both CSVs
signup_df = pd.read_csv(google_drive_data_path + '/signup_survey_data_processed_combined.csv')
survey_meta_df = df[['Email', 'status']]  # Using the df we created earlier

# Merge the dataframes
signup_df = signup_df.merge(survey_meta_df, on='Email', how='left')

# Replace values in Note column with status values where status is not null
mask = pd.notna(signup_df['status'])
signup_df.loc[mask, 'Note'] = signup_df.loc[mask, 'status']

# Drop the status column since we don't need it anymore
signup_df = signup_df.drop('status', axis=1)


In [ ]:
signup_df.to_csv(google_drive_data_path + '/signup_survey_data_with_note.csv', index=False)

In [ ]:
# Read the Participants from Janel CSV
janel_df = pd.read_csv(google_drive_data_path + '/Participants from Janel.csv')

# Merge with signup_df
janel_df = janel_df.merge(signup_df[['Email', 'Note']],
                         on='Email',
                         how='left',
                         suffixes=('_old', '_new'))

# If Note_new exists, use it to replace Note_old
mask = pd.notna(janel_df['Note_new'])
janel_df.loc[mask, 'Note_old'] = janel_df.loc[mask, 'Note_new']

# Clean up column names
janel_df = janel_df.rename(columns={'Note_old': 'Note'})
janel_df = janel_df.drop('Note_new', axis=1)

print(janel_df)

                     Name                          Email signed up?  \
0   America haileselassie   Ahaileselassie7459@gmail.com        yes   
1           Barbara Stras          Barbstraskc@gmail.com    not yet   
2           Brian Troesch          Brian_60629@yahoo.com        yes   
3         Brittany Thomas            btmarie33@gmail.com        yes   
4           David Seltzer                djsboca@aol.ocm        yes   
5                 Hong Vu              hongvuh@gmail.com        yes   
6               Art Smosh            ibbincome@gmail.com        yes   
7             Juan Moreno         jimoreno2055@gmail.com        yes   
8      Katherine Mitchell  katherinemitchell98@yahoo.com    not yet   
9        Kelley Pasmanick      kelleypasmanick@gmail.com        yes   
10       Kimberly Tippett           Ktippett05@nc.rr.com        yes   
11            Lauren Rich        laurenjrich@outlook.com        yes   
12             carl mapps            Mappscarl@gmail.com        yes   
13    

In [ ]:
janel_df.to_csv(google_drive_data_path + '/Participants from Janel noted.csv', index=False)

## Generate reminder mail list

Generate a list of emails for certain filter. Copy the output into bcc in gmail

Locate the corresponding email in the document [Emails](https://docs.google.com/document/d/1e5TZJ6ILpku8rVdO_ADUwI-lM8SCIh7xG95jrG73INY/edit?usp=sharing)

In [ ]:
def get_emails_by_day_count(day_count):
    emails = survey_meta_manager[survey_meta_manager['app_day_count'] == day_count]['Email'].tolist()
    if emails:
        return ','.join(emails)
    else:
        return None

### Week 1 completion

In [ ]:
# filter those day_count is 7
emails = get_emails_by_day_count(7)
print(emails)

None


### Week 2 completion

In [ ]:
# filter those day_count is 14
emails = get_emails_by_day_count(14)
print(emails)

None


### Week 3 completion

In [ ]:
# filter those day_count is 21
emails = get_emails_by_day_count(21)
print(emails)

None


### Daily reminder

In [ ]:
# if there is an available survey awaiting
# Create four columns with initial 'unavailable' status
survey_meta_manager['survey1_status'] = 'unavailable'
survey_meta_manager['survey2_status'] = 'unavailable'
survey_meta_manager['survey3_status'] = 'unavailable'
survey_meta_manager['survey4_status'] = 'unavailable'

# Update status to 'complete' if there's a date in the survey columns
for i in range(1, 5):
    survey_meta_manager[f'survey{i}_status'] = np.where(survey_meta_manager[f'survey{i}'].notna(), 'complete', survey_meta_manager[f'survey{i}_status'])

# Update status to 'available' if survey is NA and day_count exceeds the threshold
thresholds = [6, 13, 20]
for i, col in enumerate(['survey2', 'survey3', 'survey4'], start=2):
    survey_meta_manager[f'{col}_status'] = np.where(
        (survey_meta_manager[col].isna()) & (survey_meta_manager['app_day_count'] > thresholds[i-2]),
        'available',
        survey_meta_manager[f'{col}_status']
    )

# Save the updated DataFrame
print("The shape of the dataframe is: ", survey_meta_manager.shape)
survey_meta_manager.to_csv(google_drive_data_path + '/survey_meta_manager.csv', index=False)

The shape of the dataframe is:  (8, 13)


In [ ]:
# Get emails for participants with at least one 'available' survey
target_participants = survey_meta_manager[(survey_meta_manager['survey2_status'] == 'available') |
                                          (survey_meta_manager['survey3_status'] == 'available') |
                                          (survey_meta_manager['survey4_status'] == 'available')]
email_list = target_participants['email'].tolist()
print(','.join(email_list) or 'None')

pi005@gmail.com,pi011@gmail.com,pi012@gmail.com,te010@gmail.com


### Drop out notification

In [ ]:
# If a survey is unanswered for seven days, send out a drop-out notification